### RAG with Astra DB Vector store

In [25]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
embeddings_google = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings_google

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x1145ef290>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [3]:
len(embeddings_google.embed_query("Hello AI"))

768

In [4]:
# Google LLM
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [5]:
# Prompt
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt.messages

/Users/I572648/Library/CloudStorage/OneDrive-SAPSE/Desktop/Git/GenAI/.venv/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [26]:
ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_KEYSPACE = "samplekeyspace2"
ASTRA_DB_KEYSPACE

'samplekeyspace2'

In [27]:
from langchain_astradb import AstraDBVectorStore

vector_store_explicit_embeddings = AstraDBVectorStore(
    collection_name="astra_vector_langchain",
    embedding=embeddings_google,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE,
)

In [30]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders.text import TextLoader

loader = TextLoader("../data/sample.txt")
text_document = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)
texts = text_splitter.split_documents(text_document)
print(texts[0].page_content)
texts

Created a chunk of size 167, which is longer than the specified 100


Langchain is specially used for LLM powered applications.


[Document(metadata={'source': '../data/sample.txt'}, page_content='Langchain is specially used for LLM powered applications.'),
 Document(metadata={'source': '../data/sample.txt'}, page_content='Langchain  -> Chaining  -> Focus on Sequential Execution of process\n* In Langchain, everything happens in sequential order So we call it as Directed Acyclic Graph(DAG)'),
 Document(metadata={'source': '../data/sample.txt'}, page_content='LangGraph: We can create stateful Multi AI Agents Applications where Agents will be communicating with themselves to solve complex workflow.\n* They follow the graph structure and it is also going to maintain information which will be shared within the Nodes that is why it is called stategraph.')]

In [31]:
vector_store_explicit_embeddings.add_documents(documents=texts)

['7e4a0a06269b4f9ea726b8649a59e508',
 '476bcdaa1d0e4f8985ef3c8f7ddf3901',
 'bbf352746ead40ca8a97abc13321718c']

In [32]:
retriever = vector_store_explicit_embeddings.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)

In [33]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [35]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [36]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [37]:
rag_chain.invoke("What is LangGraph?")

'LangGraph is a system for creating stateful multi-AI agent applications.  These agents communicate to solve complex workflows, following a graph structure and sharing information between nodes.  This shared information is what makes it a "stategraph."'